In [1]:
import dask.dataframe as dd
import polars as pl

In [2]:
lista = ['CTU-Honeypot-Capture-4-1','CTU-Honeypot-Capture-5-1','CTU-Honeypot-Capture-7-1', #2
         'CTU-IoT-Malware-Capture-1-1','CTU-IoT-Malware-Capture-3-1','CTU-IoT-Malware-Capture-7-1',#5
         'CTU-IoT-Malware-Capture-8-1', 'CTU-IoT-Malware-Capture-20-1', 'CTU-IoT-Malware-Capture-21-1',#8
         'CTU-IoT-Malware-Capture-33-1', 'CTU-IoT-Malware-Capture-34-1','CTU-IoT-Malware-Capture-35-1',#11
         'CTU-IoT-Malware-Capture-36-1','CTU-IoT-Malware-Capture-42-1','CTU-IoT-Malware-Capture-44-1', #14
         'CTU-IoT-Malware-Capture-48-1','CTU-IoT-Malware-Capture-60-1'] #16

caminhos_arquivos = [f'../../{nome_arquivo}.parquet' for nome_arquivo in lista]


In [3]:
dfs = [dd.read_parquet(caminho, engine='pyarrow') for caminho in caminhos_arquivos]
df = dd.concat(dfs)

In [4]:
#df = df.sample(frac=0.01, random_state=42)
del lista, dfs,caminhos_arquivos

# Pré-processamento

In [5]:
df

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,label,detailed-label
npartitions=21,,,,,,,,,,,,,,,,,,,,,,,
,float64,string,string,int64,string,int64,string,string,object,object,object,string,string,string,int64,string,int64,int64,int64,int64,string,string,string
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [6]:
# Função para substituir "-" por "0" em uma coluna
def replace_dash_with_zero_dask(df, column):
    df[column] = df[column].replace("-", "0", regex=True)
    return df

# Aplicar a substituição nas colunas desejadas
df = df.map_partitions(replace_dash_with_zero_dask, column='duration')
df = df.map_partitions(replace_dash_with_zero_dask, column='orig_bytes')
df = df.map_partitions(replace_dash_with_zero_dask, column='resp_bytes')

# Conversão de tipos
df['ts'] = df['ts'].astype(float)
df['uid'] = df['uid'].astype(str)
df['id.orig_h'] = df['id.orig_h'].astype(str)
df['id.orig_p'] = df['id.orig_p'].astype(int)
df['id.resp_h'] = df['id.resp_h'].astype(str)
df['id.resp_p'] = df['id.resp_p'].astype(int)
df['proto'] = df['proto'].astype(str)
df['service'] = df['service'].astype(str)
df['duration'] = df['duration'].astype(float)
df['orig_bytes'] = df['orig_bytes'].astype(float)
df['resp_bytes'] = df['resp_bytes'].astype(float)
df['conn_state'] = df['conn_state'].astype(str)
df['local_orig'] = df['local_orig'].astype(str)
df['local_resp'] = df['local_resp'].astype(str)
df['missed_bytes'] = df['missed_bytes'].astype(int)
df['history'] = df['history'].astype(str)
df['tunnel_parents'] = df['tunnel_parents'].astype(str)
df['label'] = df['label'].astype(str)

In [7]:
def ipv4_to_int(ip):
    if not isinstance(ip, str):  # Verifica se o valor é uma string válida
        return None
    try:
        parts = [int(part) for part in ip.split('.')]
        return (parts[0] << 24) + (parts[1] << 16) + (parts[2] << 8) + parts[3]
    except Exception:
        return None  # Retorna None se ocorrer algum problema

In [8]:
df['id.resp_h'] = df['id.resp_h'].apply(ipv4_to_int)
df['id.orig_h'] = df['id.resp_h'].apply(ipv4_to_int)

c:\Users\rafae\miniconda3\envs\env-tcc\lib\site-packages\dask_expr\_collection.py:4341: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('id.resp_h', 'object'))

  warnings.warn(meta_warning(meta))
c:\Users\rafae\miniconda3\envs\env-tcc\lib\site-packages\dask_expr\_collection.py:4341: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=

In [9]:
# Mapear labels
df['label'] = df['label'].str.replace('benign', 'Benign')
df['label'] = df['label'].map({'Benign': 0, 'Malicious': 1})
df['label'] = df['label'].astype(int)

c:\Users\rafae\miniconda3\envs\env-tcc\lib\site-packages\dask_expr\_collection.py:4160: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('label', 'float64'))

  warnings.warn(meta_warning(meta))


In [10]:
label_columns = ['proto', 'service', 'conn_state', 'history']

def label_encode_column(df, col):
    unique_values = df[col].dropna().unique()
    label_mapping = {value: i for i, value in enumerate(unique_values)}
    df[col] = df[col].map(label_mapping)
    return df

for col in label_columns:
    df = df.map_partitions(label_encode_column, col=col)

: 

In [ ]:
pandas_df = df.compute()

C:\Users\rafae\AppData\Local\Temp\ipykernel_10372\3648796677.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[column] = df[column].replace("-", "0", regex=True)
C:\Users\rafae\AppData\Local\Temp\ipykernel_10372\3648796677.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[column] = df[column].replace("-", "0", regex=True)
C:\Users\rafae\AppData\Local\Temp\ipykernel_10372\3648796677.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, exp

In [ ]:
df_polars = pl.from_pandas(pandas_df)

del pandas_df

In [ ]:
df_polars

In [ ]:
df_polars.write_parquet('dataset2.parquet', compression='snappy')
print(f"Arquivo foi salvo com sucesso.")

In [ ]:
#arquivo = "dataset"
#df_pandas.to_parquet(f'{arquivo}.parquet', compression='snappy')

#print(f"Arquivo '{arquivo}.parquet' salvo com sucesso.")